### JSON Objects in PostgreSQL

**Author**: Aaron Liu & Ron Volkovinsky

**Date**: 5/10/2023

**Objective**: Practice basic insert queries using SQL, and the corresponding programming to automate the process in psycopg2

## Setup

Some functions in python are already given in the cells below:

**DOI JSON retrieval (doi2dict)**
(Credit Ron Volkovinsky)

In [1]:
import requests
import json
import pandas as pd
import bibtexparser
import pprint

def doi2dict(doi):
    #create url
    url = "http://dx.doi.org/" + doi
    
    #create dictionary of http bibtex headers that requests will retrieve from the url
    headers = {"accept": "application/x-bibtex"}
    
    #reqeusts information specified by bibtex from url
    r = requests.get(url, headers = headers).text    

    #parse the returned bibtex text to a dictionary
    #NOTE: USE bibtexparser.customization to split strings into list, etc. (https://bibtexparser.readthedocs.io/en/master/bibtexparser.html?highlight=bparser#module-bibtexparser.bparser)
    bibdata = bibtexparser.bparser.BibTexParser().parse(r)
    
    #return dict of metadata
    return bibdata.entries[0]

doi = '10.1021/acsami.1c20994'
doi2 = '10.1021/acscentsci.9b00476'

doidict = doi2dict(doi2)

In [2]:
doidict

{'journal': '{ACS} Central Science',
 'title': '{BigSMILES}: A Structurally-Based Line Notation for Describing Macromolecules',
 'author': 'Tzyy-Shyang Lin and Connor W. Coley and Hidenobu Mochigase and Haley K. Beech and Wencong Wang and Zi Wang and Eliot Woods and Stephen L. Craig and Jeremiah A. Johnson and Julia A. Kalow and Klavs F. Jensen and Bradley D. Olsen',
 'pages': '1523--1531',
 'number': '9',
 'volume': '5',
 'publisher': 'American Chemical Society ({ACS})',
 'month': 'sep',
 'year': '2019',
 'url': 'https://doi.org/10.1021%2Facscentsci.9b00476',
 'doi': '10.1021/acscentsci.9b00476',
 'ENTRYTYPE': 'article',
 'ID': 'Lin_2019'}

**Connection Details**

Fill in your connection details here. Note that `127.0.0.1`, `localhost`, and your **local IP address** (found using the `ipconfig` command in your command line) are all synonymous with your local computer as a server. If you are connecting to an external server, you of course need to find the appropriate connection details of that server.

I recommend creating your own database as a test environment for interacting with your database. You must do this either through psql or pgAdmin, externally from Python. Call the database whatever you want, like `pg_practice` or `ofetdb_testenv`, etc. Either way, the default username and password are what go into the connection details. The port by default for PostgreSQL is almost always `5432`, unless this was specified differently during your installation of PostgreSQL.

In [3]:
import psycopg2 as pg
import sys

conn_kwargs = {
    "host"      : "localhost",
    "database"  : "test", ## FILL IN CONNECTION DETAILS HERE
    "user"      : "postgres",
    "password"  : "myL220q1W@",
    "port"      : "5432",
}

def connect(**params_dict):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = pg.connect(**params_dict)
    except (Exception, pg.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

conn = connect(**conn_kwargs)

conn.close()
print("Connection Closed")

Connecting to the PostgreSQL database...
Connection successful
Connection Closed


In [4]:
# Postgres python
import psycopg2 as pg
import numpy as np
from psycopg2.extensions import AsIs

# import os
# import functools
# import sys

# Adapters necessary for converting python data types to PostgreSQL compatible data types 
def addapt_numpy_float64(numpy_float64):
    return AsIs(numpy_float64)

def addapt_numpy_int64(numpy_int64):
    return AsIs(numpy_int64)

def nan_to_null(f,
        _NULL=AsIs('NULL'),
        _Float=pg.extensions.Float):
    if not np.isnan(f):
        return _Float(f)
    return _NULL

pg.extensions.register_adapter(np.float64, addapt_numpy_float64)
pg.extensions.register_adapter(np.int64, addapt_numpy_int64)
pg.extensions.register_adapter(float, nan_to_null)

def pg_query(sql, tup):
    
    try:
        # Database connection
        conn = pg.connect(**conn_kwargs)
        cur = conn.cursor()
        
        # Pass SQL query, using string and placeholders
        cur.execute(sql, tup)
        
#         # Fetch result
#         fetched = cur.fetchone()[0]
        
        # Commit result
        conn.commit()
        print("Operation Successful")

        cur.close()
        conn.close()
        
    except (Exception, pg.DatabaseError) as error:
        # If database connection unsuccessful, then close connection 
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        conn.close()
    
    return 

In [29]:
# Create a table that holds journal article information
conn = connect(**conn_kwargs)

cur = conn.cursor()

sql = '''

    DROP TABLE IF EXISTS EXPERIMENT_INFO;

    CREATE TABLE EXPERIMENT_INFO (
        exp_id              SERIAL          PRIMARY KEY,
        citation_type       VARCHAR(20),
        meta                JSONB,
        UNIQUE(citation_type, meta)
    );
'''

cur.execute(sql)
conn.commit()

print("Operation successful")
conn.close()

Connecting to the PostgreSQL database...
Connection successful
Operation successful


In [6]:
doidict

{'journal': '{ACS} Central Science',
 'title': '{BigSMILES}: A Structurally-Based Line Notation for Describing Macromolecules',
 'author': 'Tzyy-Shyang Lin and Connor W. Coley and Hidenobu Mochigase and Haley K. Beech and Wencong Wang and Zi Wang and Eliot Woods and Stephen L. Craig and Jeremiah A. Johnson and Julia A. Kalow and Klavs F. Jensen and Bradley D. Olsen',
 'pages': '1523--1531',
 'number': '9',
 'volume': '5',
 'publisher': 'American Chemical Society ({ACS})',
 'month': 'sep',
 'year': '2019',
 'url': 'https://doi.org/10.1021%2Facscentsci.9b00476',
 'doi': '10.1021/acscentsci.9b00476',
 'ENTRYTYPE': 'article',
 'ID': 'Lin_2019'}

In [7]:
from psycopg2.extras import Json

doi = '10.1021/acsami.1c20994'
doi2 = '10.1021/acscentsci.9b00476'

doidict = doi2dict(doi)

a = Json(doidict)

print(a)

'{"journal": "{ACS} Applied Materials {\\&}amp$\\mathsemicolon$ Interfaces", "title": "The Solution is the Solution: Data-Driven Elucidation of Solution-to-Device Feature Transfer for $\\uppi$-Conjugated Polymer Semiconductors", "author": "Connor P. Callaway and Aaron L. Liu and Rahul Venkatesh and Yulong Zheng and Myeongyeon Lee and J. Carson Meredith and Martha Grover and Chad Risko and Elsa Reichmanis", "pages": "3613--3620", "number": "3", "volume": "14", "publisher": "American Chemical Society ({ACS})", "month": "jan", "year": "2022", "url": "https://doi.org/10.1021%2Facsami.1c20994", "doi": "10.1021/acsami.1c20994", "ENTRYTYPE": "article", "ID": "Callaway_2022"}'


In [8]:
import psycopg2

kwargs = {
    'database': 'test',
    'user': 'postgres',
    'password': 'myL220q1W@',
    'host': '127.0.0.1',
    'port': '5432'
}

# %% Create Tables for EXPERIMENT_INFO

conn = psycopg2.connect(**kwargs)

print("Connection Successful")

cur = conn.cursor()
cur.execute(
    '''
    CREATE TABLE IF NOT EXISTS EXPERIMENT_INFO (
        exp_id              SERIAL          PRIMARY KEY,
        citation_type       VARCHAR(20),
        meta                JSONB,
        UNIQUE(citation_type, meta)
    );
    '''
)

print("Table(s) created successfully")
conn.commit()

print("Operation successful")
conn.close()

Connection Successful
Table(s) created successfully
Operation successful


In [9]:
Json(doidict)

In [10]:
sql = "INSERT INTO experiment_info(%s) VALUES %s"

columns = ['citation_type', 'meta']
values = ['literature', Json(doidict)]

tup = (AsIs(','.join(columns)), tuple(values))

conn = pg.connect(**kwargs)

print("Connection Successful")

cur = conn.cursor()
cur.execute(sql, tup)

print("Table(s) created successfully")
conn.commit()

print("Operation successful")
conn.close()


Connection Successful
Table(s) created successfully
Operation successful


### Exercise

Populate a journal article DOI table

1. Create a table called Articles. We want the table to contain columns that store a unique id for each row, the year it was published, and a metadata field with variable information mined using bibtexparser
2. Populate the table with all the information using the Excel file "articles.xlsx" provided

In [11]:
import pandas as pd
df = pd.read_excel('articles.xlsx')
df

,doi
0,10.1002/adfm.201002729
1,10.1021/am3027822
2,10.1016/j.orgel.2011.06.027
3,10.1021/cm049617w
4,10.1103/PhysRevB.74.115318
5,10.1021/nn401323f
6,10.1063/1.2400796
7,10.1002/adfm.201403708
8,10.1002/adfm.201403708
9,10.1007/s11801-011-0122-z


In [12]:
df.doi[0]

'10.1002/adfm.201002729'

In [13]:
type(df.doi[0])

str

In [30]:
conn = connect(**conn_kwargs)

cur = conn.cursor()

sql = '''

    DROP TABLE IF EXISTS Articles;

    CREATE TABLE Articles (
        exp_id              SERIAL          PRIMARY KEY,
        citation_type       VARCHAR(20),
        meta                JSONB,
        UNIQUE(citation_type, meta)
    );
'''

cur.execute(sql)
conn.commit()

print("Operation successful")
conn.close()

Connecting to the PostgreSQL database...
Connection successful
Operation successful


In [58]:
#Failed

for row in df.iterrows():
    doidict = doi2dict(row)
    print(dodict)
 TypeError: can only concatenate str (not "tuple") to str

for row in df.itertuples():
    doidict = doi2dict(row)
    print(doidict)
 TypeError: can only concatenate str (not "Pandas") to str

for row in df.iterrows():
    doidict = doi2dict(row[1])
    print(dodict)
InvalidSchema: No connection adapters were found for 'doi  http://dx.doi.org/10.1002/adfm.201002729\nName: 0, dtype: object'

{'journal': 'Advanced Functional Materials', 'title': 'Tunable Crystallinity in Regioregular Poly(3-Hexylthiophene) Thin Films and Its Impact on Field Effect Mobility', 'author': 'Avishek R. Aiyar and Jung-Il Hong and Rakesh Nambiar and David M. Collard and Elsa Reichmanis', 'pages': '2652--2659', 'number': '14', 'volume': '21', 'publisher': 'Wiley', 'month': 'may', 'year': '2011', 'url': 'https://doi.org/10.1002%2Fadfm.201002729', 'doi': '10.1002/adfm.201002729', 'ENTRYTYPE': 'article', 'ID': 'Aiyar_2011'}
{'journal': '{ACS} Applied Materials {\\&}amp$\\mathsemicolon$ Interfaces', 'title': 'Ultrasound-Induced Ordering in Poly(3-hexylthiophene): Role of Molecular and Process Parameters on Morphology and Charge Transport', 'author': 'Avishek R. Aiyar and Jung-Il Hong and Jessica Izumi and Dalsu Choi and Nabil Kleinhenz and Elsa Reichmanis', 'pages': '2368--2377', 'number': '7', 'volume': '5', 'publisher': 'American Chemical Society ({ACS})', 'month': 'mar', 'year': '2013', 'url': 'https

{'journal': 'Journal of Polymer Science Part B: Polymer Physics', 'title': 'Control of aggregate formation in poly(3-hexylthiophene) by solvent, molecular weight, and synthetic method', 'author': 'Christina Scharsich and Ruth H. Lohwasser and Michael Sommer and Udom Asawapirom and Ullrich Scherf and Mukundan Thelakkat and Dieter Neher and Anna KÃ¶hler', 'pages': '442--453', 'number': '6', 'volume': '50', 'publisher': 'Wiley', 'month': 'dec', 'year': '2011', 'url': 'https://doi.org/10.1002%2Fpolb.23022', 'doi': '10.1002/polb.23022', 'ENTRYTYPE': 'article', 'ID': 'Scharsich_2011'}
{'journal': 'Journal of Polymer Science Part B: Polymer Physics', 'title': 'Control of aggregate formation in poly(3-hexylthiophene) by solvent, molecular weight, and synthetic method', 'author': 'Christina Scharsich and Ruth H. Lohwasser and Michael Sommer and Udom Asawapirom and Ullrich Scherf and Mukundan Thelakkat and Dieter Neher and Anna KÃ¶hler', 'pages': '442--453', 'number': '6', 'volume': '50', 'publi

SSLError: HTTPSConnectionPool(host='dx.doi.org', port=443): Max retries exceeded with url: /10.1002/adfm.200400017 (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1129)')))

In [93]:
for row in df.itertuples():
    doidict = doi2dict(row[1])
    print(doidict)

{'journal': 'Advanced Functional Materials', 'title': 'Tunable Crystallinity in Regioregular Poly(3-Hexylthiophene) Thin Films and Its Impact on Field Effect Mobility', 'author': 'Avishek R. Aiyar and Jung-Il Hong and Rakesh Nambiar and David M. Collard and Elsa Reichmanis', 'pages': '2652--2659', 'number': '14', 'volume': '21', 'publisher': 'Wiley', 'month': 'may', 'year': '2011', 'url': 'https://doi.org/10.1002%2Fadfm.201002729', 'doi': '10.1002/adfm.201002729', 'ENTRYTYPE': 'article', 'ID': 'Aiyar_2011'}
{'journal': '{ACS} Applied Materials {\\&}amp$\\mathsemicolon$ Interfaces', 'title': 'Ultrasound-Induced Ordering in Poly(3-hexylthiophene): Role of Molecular and Process Parameters on Morphology and Charge Transport', 'author': 'Avishek R. Aiyar and Jung-Il Hong and Jessica Izumi and Dalsu Choi and Nabil Kleinhenz and Elsa Reichmanis', 'pages': '2368--2377', 'number': '7', 'volume': '5', 'publisher': 'American Chemical Society ({ACS})', 'month': 'mar', 'year': '2013', 'url': 'https

{'journal': 'Journal of Polymer Science Part B: Polymer Physics', 'title': 'Control of aggregate formation in poly(3-hexylthiophene) by solvent, molecular weight, and synthetic method', 'author': 'Christina Scharsich and Ruth H. Lohwasser and Michael Sommer and Udom Asawapirom and Ullrich Scherf and Mukundan Thelakkat and Dieter Neher and Anna KÃ¶hler', 'pages': '442--453', 'number': '6', 'volume': '50', 'publisher': 'Wiley', 'month': 'dec', 'year': '2011', 'url': 'https://doi.org/10.1002%2Fpolb.23022', 'doi': '10.1002/polb.23022', 'ENTRYTYPE': 'article', 'ID': 'Scharsich_2011'}
{'journal': 'Journal of Polymer Science Part B: Polymer Physics', 'title': 'Control of aggregate formation in poly(3-hexylthiophene) by solvent, molecular weight, and synthetic method', 'author': 'Christina Scharsich and Ruth H. Lohwasser and Michael Sommer and Udom Asawapirom and Ullrich Scherf and Mukundan Thelakkat and Dieter Neher and Anna KÃ¶hler', 'pages': '442--453', 'number': '6', 'volume': '50', 'publi

In [16]:
sql = """INSERT INTO Articles(%s) VALUES %s"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    
    cur = conn.cursor()
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")

conn.close()

UniqueViolation: 오류:  중복된 키 값이 "articles_citation_type_meta_key" 고유 제약 조건을 위반함
DETAIL:  (citation_type, meta)=(literature, {"ID": "Choi_2014", "doi": "10.1002/adfm.201403708", "url": "https://doi.org/10.1002%2Fadfm.201403708", "year": "2014", "month": "dec", "pages": "920--927", "title": "Controlled Assembly of Poly(3-hexylthiophene): Managing the Disorder to Order Transition on the Nano- through Meso-Scales", "author": "Dalsu Choi and Mincheol Chang and Elsa Reichmanis", "number": "6", "volume": "25", "journal": "Advanced Functional Materials", "ENTRYTYPE": "article", "publisher": "Wiley"}) 키가 이미 있습니다.


In [94]:
#Failed

sql = """INSERT INTO Articles(%s) VALUES %s

ON CONFLICT (citation_type) DO UPDATE
    SET citation_type = excluded.citation_type,
        meta = excluded.meta"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    
    cur = conn.cursor()
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")

conn.close()

# Error: when use "ON CONFLICT", unique or exclude conditoin shoul existe

InvalidColumnReference: 오류:  ON CONFLICT 절을 사용하는 경우, unique 나 exclude 제약 조건이 있어야 함


In [92]:
#Failed

sql = """INSERT INTO Articles(%s) VALUES %s

ON CONFLICT (exp_id) DO UPDATE
    SET citation_type = excluded.citation_type,
        meta = excluded.meta"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    cur = conn.cursor()
    
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")

conn.close()

# result is weired, exp_id is started at #33

InvalidColumnReference: 오류:  ON CONFLICT 절을 사용하는 경우, unique 나 exclude 제약 조건이 있어야 함


In [99]:
sql = """INSERT INTO Articles(%s) VALUES %s

ON CONFLICT (meta) DO UPDATE
    SET citation_type = excluded.citation_type,
        meta = excluded.meta"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    
    cur = conn.cursor()
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")
conn.close()

# Error: when use "ON CONFLICT", unique or exclude conditoin shoul existe

InvalidColumnReference: 오류:  ON CONFLICT 절을 사용하는 경우, unique 나 exclude 제약 조건이 있어야 함


In [25]:
sql = """INSERT INTO Articles(%s) VALUES %s

ON CONFLICT (citation_type, meta) DO UPDATE
    SET citation_type = excluded.citation_type,
        meta = excluded.meta"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    
    cur = conn.cursor()
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")
conn.close()

# exp_id number don't start at 1

Connection Successful
Table(s) created successfully
Operation successful


In [31]:
sql = """INSERT INTO Articles(%s) VALUES %s

ON CONFLICT (citation_type, meta) DO UPDATE
    SET citation_type = excluded.citation_type,
        meta = excluded.meta"""
    
for row in df.itertuples():
    doidict = doi2dict(row[1])
    columns = ['citation_type', 'meta']
    values = ['literature', Json(doidict)]
    tup = (AsIs(','.join(columns)), tuple(values))

    conn = pg.connect(**kwargs)
    
    cur = conn.cursor()
    cur.execute(sql, tup)
    conn.commit()
    
print("Connection Successful")
print("Table(s) created successfully")
print("Operation successful")
conn.close()


Connection Successful
Table(s) created successfully
Operation successful


In [ ]:
## I left off talking about inserting new tuples that already exist... and violating key constraints. What about sequencing?
## Let's insert like 5 doi's, see what happens